In [1]:
import torch
import torchaudio
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn # 인공 신경망 모델들 모아놓은 모듈
import torch.nn.functional as F #그중 자주 쓰이는것들을 F로
from torchvision import transforms, datasets
import cv2
from torchvision import transforms, datasets
import pandas as pd
import os
from glob import glob
import torchvision.models as models
import sys
import math



p = os.path.abspath('../') # 상위 폴더를 사용하기 위해서.
sys.path.insert(1, p)
from pytorchtools.pytorchtools import EarlyStopping # 상위 폴더에 추가된 모듈.

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
#DEVICE = torch.device('cpu')
print('Using Pytorch version : ',torch.__version__,' Device : ',DEVICE)

Using Pytorch version :  1.10.2  Device :  cuda


In [2]:
import numpy as np
import librosa, librosa.display 
import matplotlib.pyplot as plt
#window sizde : FFT를 할때 참조할 그래프 길이 ( 프레임 하나당 sample 수 )
#자연어 처리에서는 25ms 사용. https://ahnjg.tistory.com/93
#초당 50000hz 중 1250개씩 윈도우 사이즈로 사용.
sr=50000
win_length =  np.int64(50000/40) # 1250
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.

# data 분류

In [3]:
#1. train, test 나누기

import os
from glob import glob
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.model_selection import train_test_split # train , test 분리에 사용.


pathology = glob('../../voice_data/fusion/pathology/phrase/*.wav')
healthy = glob('../../voice_data/fusion/healthy/phrase/*.wav')
print("Pathology : ",len(pathology))
print("Healthy: ",len(healthy))


pathology= [ path.split("\\")[-1] for path in pathology] # path 데이터 변환.
healthy= [ path.split("\\")[-1] for path in healthy] # path 데이터 변환.
 # path 데이터 변환


X = pathology+healthy # path 데이터 합
print("총 데이터수 : ",len(X))
Y = [] # 라벨
for idx,x in enumerate(X):
    if idx<1193:
        Y.append("pathology")
    else:
        Y.append("healthy")

X, X_test, Y, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, stratify=Y, random_state=456)
#stratify를 넣어서, test에도 라벨별 잘 분류되게 한다.

print("---")
print("훈련 셋 : ",len(Y),Counter(Y))
print("테스트 셋 : ",len(Y_test),Counter(Y_test))
print("---")

Pathology :  1193
Healthy:  634
총 데이터수 :  1827
---
훈련 셋 :  1461 Counter({'pathology': 954, 'healthy': 507})
테스트 셋 :  366 Counter({'pathology': 239, 'healthy': 127})
---


In [4]:
#1. train, test 나누기
#stratified kfold
import os
import random #데이터 shuffle 사용
from glob import glob
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from collections import Counter



skf_iris = StratifiedKFold(n_splits=5,shuffle=True,random_state=456)
cnt_iter = 0

X_train_list = [] #데이터 셋 보관
Y_train_list = []

X_valid_list = []
Y_valid_list = []

for train_idx, test_idx in skf_iris.split(X,Y):
    
    #split으로 반환된 인덱스를 이용하여, 학습 검증용 테스트 데이터 추출
    cnt_iter += 1
    X_train, X_valid = [X[idx] for idx in train_idx.tolist() ], [X[idx] for idx in test_idx.tolist() ]
    Y_train, Y_valid = [Y[idx] for idx in train_idx.tolist() ], [Y[idx] for idx in test_idx.tolist() ]
    
    X_train_list.append(X_train)
    X_valid_list.append(X_valid)
    
    Y_train_list.append(Y_train)
    Y_valid_list.append(Y_valid)
    
    
    #학습 및 예측
    
    label_train = Y_train
    label_test = Y_valid
    unique_train, train_counts = np.unique(label_train, return_counts = True)
    unique_test, test_counts = np.unique(label_test, return_counts = True)
    
    uniq_cnt_train = dict(zip(unique_train, train_counts))
    uniq_cnt_test = dict(zip(unique_test, test_counts))
    
    
    
    print('교차 검증 : {}'.format(cnt_iter))
    print('학습 레이블 데이터 분포 : \n', uniq_cnt_train)
    print('검증 레이블 데이터 분포 : \n', uniq_cnt_test,'\n')



교차 검증 : 1
학습 레이블 데이터 분포 : 
 {'healthy': 405, 'pathology': 763}
검증 레이블 데이터 분포 : 
 {'healthy': 102, 'pathology': 191} 

교차 검증 : 2
학습 레이블 데이터 분포 : 
 {'healthy': 406, 'pathology': 763}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 191} 

교차 검증 : 3
학습 레이블 데이터 분포 : 
 {'healthy': 406, 'pathology': 763}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 191} 

교차 검증 : 4
학습 레이블 데이터 분포 : 
 {'healthy': 406, 'pathology': 763}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 191} 

교차 검증 : 5
학습 레이블 데이터 분포 : 
 {'healthy': 405, 'pathology': 764}
검증 레이블 데이터 분포 : 
 {'healthy': 102, 'pathology': 190} 



In [5]:
#2. random over sampling
for i in range(5):
    X_temp = np.array(X_train_list[i]).reshape(-1,1)#각 데이터를 다 행으로 넣음. (1194,1)
    #Y = np.array(Y)
    ros = RandomOverSampler(random_state = 123)
    X_res,Y_res = ros.fit_resample(X_temp,Y_train_list[i])
    
    print("\n fold{} ".format(i))
    print('before dataset shape {}'.format(Counter(Y_train_list[i])) )
    print('Resampled dataset shape {}'.format(Counter(Y_res)) )
    
    #원래대로 돌리기
    X_res=X_res.reshape(1, -1)
    X_train_list[i]=X_res[0].tolist()
    Y_train_list[i]=Y_res




 fold0 
before dataset shape Counter({'pathology': 763, 'healthy': 405})
Resampled dataset shape Counter({'pathology': 763, 'healthy': 763})

 fold1 
before dataset shape Counter({'pathology': 763, 'healthy': 406})
Resampled dataset shape Counter({'pathology': 763, 'healthy': 763})

 fold2 
before dataset shape Counter({'pathology': 763, 'healthy': 406})
Resampled dataset shape Counter({'pathology': 763, 'healthy': 763})

 fold3 
before dataset shape Counter({'pathology': 763, 'healthy': 406})
Resampled dataset shape Counter({'pathology': 763, 'healthy': 763})

 fold4 
before dataset shape Counter({'pathology': 764, 'healthy': 405})
Resampled dataset shape Counter({'pathology': 764, 'healthy': 764})


In [6]:
import pickle

 
#load
with open("../../voice_data/fusion/phrase_sig_dict.pickle","rb") as fr:
    phrase_dict = pickle.load(fr)

    

# data set

In [7]:
from torch.utils.data import Dataset, DataLoader

classes = ["pathology","healthy"]
sr=16000
win_length =  np.int64(sr/40) # 1250
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.



class svd_dataset(Dataset):
    def __init__(self,data_path_list,classes,data_num,training):
        #클래스에서 사용할 인자를 받아 인스턴스 변수로 저장하는 일을 한다.
        #예를들면, 이미지의 경로 리스트를 저장하는 일을 하게 된다.
        
        #data_num : k 개 데이터 셋 중 어떤것을 쓸지
        #test인지 아닌지.
        
        self.path_list = data_path_list[data_num]
        self.data_num = data_num
        self.training = training
        self.label = svd_dataset.get_label(self.path_list,training,data_num)
        self.classes=classes
        
    
    @classmethod
    def get_label(cls,data_path_list,training,data_num):
        label_list=[]
        
        if training:
            for idx,x in enumerate(data_path_list):
                label_list.append(Y_train_list[data_num][idx])
        else:
            for idx,x in enumerate(data_path_list):
                label_list.append(Y_valid_list[data_num][idx])
        #print(label_list)
        return label_list
    
    def __len__(self):
        return len(self.path_list)
        #데이터 셋의 길이를 정수로 반환한다.     
        
    def __getitem__(self, idx):
        """
        1. path를 받아서, signal 변환
        2. raw 데이터 바로 출력.
        
        """

        sig = phrase_dict[self.path_list[idx]]
        pad1d=lambda a, i: a[0:i] if a.shape[0] > i else np.hstack((a, np.zeros((i-a.shape[0]))))
        length = 101951
        sig=pad1d(sig,length)
        sig=torch.from_numpy(sig).type(torch.float32)# 타입 변화
        sig=sig.unsqueeze(0)
        
        return sig, self.classes.index(self.label[idx])

In [8]:
# test set 제작을 위한 class
class svd_test_set(Dataset):
    def __init__(self,data_path_list,classes):
        #클래스에서 사용할 인자를 받아 인스턴스 변수로 저장하는 일을 한다.
        #예를들면, 이미지의 경로 리스트를 저장하는 일을 하게 된다.
        
        #data_num : k 개 데이터 셋 중 어떤것을 쓸지
        #test인지 아닌지.
        
        self.path_list = data_path_list
        self.label = svd_test_set.get_label(self.path_list)
        self.classes=classes

        
    
    @classmethod
    def get_label(cls,data_path_list):
        label_list=[]
        
        for idx,x in enumerate(data_path_list):
            label_list.append(Y_test[idx])
        #print(label_list)
        return label_list
    
    def __len__(self):
        return len(self.path_list)
        #데이터 셋의 길이를 정수로 반환한다. 
        
    def __getitem__(self, idx):
        """
        1. path를 받아서, signal 변환
        2. raw 데이터 바로 출력.
        
        """

        sig = phrase_dict[self.path_list[idx]]
        pad1d=lambda a, i: a[0:i] if a.shape[0] > i else np.hstack((a, np.zeros((i-a.shape[0]))))
        length = 101951
        sig=pad1d(sig,length)
        sig=torch.from_numpy(sig).type(torch.float32)# 타입 변화
        sig=sig.unsqueeze(0)
        
        return sig, self.classes.index(self.label[idx])

# data loader

In [9]:
#3. 하이퍼 파라미터
BATCH_SIZE =  32 #한 배치당 32개 음성데이터
EPOCHS = 50 # 전체 데이터 셋을 50번 반복

In [10]:
#DATA LOADER 함수가 BATCH_size 단위로 분리해 지정.

#확인을 위해 데이터셋 하나만 확인



train_loader = torch.utils.data.DataLoader(dataset = 
                                           svd_dataset(
                                               X_train_list,
                                                   classes,
                                               data_num=0,
                                               training=True
                                           ),
                                           batch_size = BATCH_SIZE,
                                           shuffle = True,
                                           ) # 순서가 암기되는것을 막기위해.

validation_loader = torch.utils.data.DataLoader(dataset = 
                                           svd_dataset(
                                               X_valid_list,
                                               classes,
                                               data_num=0,
                                               training=False
                                           ),
                                           batch_size = BATCH_SIZE,
                                           shuffle = True,) 



In [11]:
# 테스트 데이터 로더.

test_loader = torch.utils.data.DataLoader(dataset = 
                                               svd_test_set(
                                                   X_test,
                                                   classes,
                                               ),
                                               batch_size = BATCH_SIZE,
                                               shuffle = True,) 

# model

- https://github.com/mravanelli/SincNet/blob/master/dnn_models.py

In [12]:
###
sr=16000
win_length =  np.int64(sr/40) #
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.



# first layer(sinc layer)
class SincConv_fast(nn.Module):
    
    #필터 제작을 위한 mel 변환
    @staticmethod
    def to_mel(hz):
        return 2595*np.log10(1+hz/700)
    
    @staticmethod
    def to_hz(mel):
        return 700*(10**(mel/2595)-1)
    
    def __init__(self,out_channels,kernel_size,sample_rate=16000,in_channels=1,
                 stride=1, padding=0,dilation=1,bias=False,groups=1,min_low_hz=50,min_band_hz=50):
        
        super(SincConv_fast,self).__init__()
        
        if in_channels != 1:
            msg = "SincConv는 1차원 데이터만 입력이 가능하다. (here, in_channels={%i})"%(in_channels)
            raise ValueError(msg)
            
        
        #커널 사이즈 홀수로 만들어주기
        if kernel_size %2 == 0:
            self.kernel_size=self.kernel_size+1
                
        if bias:
            raise ValueError('SincConv does not support bias.')
        if groups > 1:
            raise ValueError('SincConv does not support groups.')
            
            
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.sample_rate = sample_rate #  input signal의 sampling rate / sincnet이 주로보는 주파수 영역대가 된다.
        self.min_low_hz = min_low_hz # f1 제작에 사용
        self.min_band_hz = min_band_hz # f2  제작에 사용
        
        #filter bank를 이용한 초기화. mel-space에서 일정한 크기.
        low_hz = 30
        high_hz = self.sample_rate/2 - (self.min_low_hz + self.min_band_hz)
        
        mel = np.linspace(self.to_mel(low_hz),
                          self.to_mel(high_hz),
                          self.out_channels+1) # 81개로 만들었다가 마지막 값을 버린다. band_hz구하려고 81칸
        hz = self.to_hz(mel)
        
        
        #filter lower frequency (out_channels,1) . f1,f2 구하는데 필요한 파라미터들
        #learnable parameter 선언
        self.low_hz_ = torch.nn.Parameter(torch.tensor(hz[:-1]).view(-1,1)) # f1 low cutoff 구현에 사용.
        self.band_hz_ = torch.nn.Parameter(torch.tensor(np.diff(hz)).view(-1,1)) # f1를 구하기위해 차를 구해둔것.        
        
        #Hamming window 파라미터들
        #hamming 윈도우 선언
        #컨볼루션 커널 절반에 해당하는 윈도우를 만들고, 대칭할 것.
        n_lin = torch.linspace(0,(self.kernel_size/2)-1,steps=int((self.kernel_size/2))) # steps : 텐서의 크기. 커널크기 반 만큼 만든다.
        # 1~커널크기 반 만큼 등차수열 만든것.
        
        self.window_ = 0.54 - 0.46*torch.cos(2*torch.pi*n_lin/self.kernel_size) # 해밍 윈도우 반.
        
        n = (self.kernel_size -1 )/2.0
        self.n_=2*torch.pi*torch.arange(-n,0).view(1,-1)/self.sample_rate # 2pi*t구현을 위한것. sinc함수에 들어간다. 음수인 이유는 절반이라서.
        # n/f=t: 시간으로. 2pi * t         
        
        
    def forward(self,waveforms):
        """f
        파라미터:
        waveforms : 'torch.Tensor' (batch_size,1,n_samples)
        
        return:
        features : 'torch.Tensor' (batch_size, out_channels, n_samples_out)
        sinc filter의 activation map이 출력된다.
        
        """
        
        
        self.n_ = self.n_.to(waveforms.device)
        self.window_ = self.window_.to(waveforms.device)
        
        # 컷오프 주파수 f1 , f2 만들기
        low = (self.min_low_hz + torch.abs(self.low_hz_)).float() # f1 구하기. 논문과는 다르다.

        #f2 구하기,
        high = torch.clamp(input=low+self.min_band_hz+torch.abs(self.band_hz_),
                           min = self.min_low_hz,
                           max = self.sample_rate/2).float() # 나이퀴스트 정리.
        band = (high-low)[:,0]#뒤에 사용하는, 싱크함수를 대칭으로 만들기 위해 정의한 것.
        
        
        # 싱크함수 만들기

        #내부 파라미터 먼저.
        f_times_t_low = torch.matmul(low,self.n_) # [80, 커널사이즈 반]
        f_times_t_high = torch.matmul(high,self.n_)
        
        #싱크함수 계산 * 해밍 윈도우
        band_pass_left = ((2*torch.sin(f_times_t_high)-2*torch.sin(f_times_t_low))/(self.n_)) * self.window_

        band_pass_center = 2*band.view(-1,1)
        band_pass_right = torch.flip(band_pass_left,dims=[1])

        band_pass = torch.cat([band_pass_left,band_pass_center,band_pass_right],dim=1)
        band_pass = band_pass / (2*band[:,None])
        #최종 밴드 필터
        self.filters = (band_pass).view(self.out_channels,
                                        1,self.kernel_size) # 중간에 1인 이유는 1d라서. [필터수,1,커널사이즈]
        
        return F.conv1d(waveforms,self.filters,stride = self.stride,
                         padding = self.padding, dilation=self.dilation,
                         bias = None, groups =1)
        

        
#### 레이어 노멀라이제이션
# 각 샘플마다 평균, 분산으로 표준화 하는 것,
# -1번째만 하는것 봐서는 
class LayerNorm(nn.Module):
    def __init__(self,features,eps=1e-6):
        super(LayerNorm,self).__init__()
        self.gamma = nn.Parameter(torch.ones(features))
        self.beta = nn.Parameter(torch.ones(features))
        #gamma와 beta는 학습된다.
        self.eps = eps
        
    def forward(self,x):
        mean = x.mean(-1,keepdim=True)
        std = x.std(-1,keepdim=True)
        return self.gamma * (x-mean)/(std + self.eps) + self.beta
        



        
class SincNet(nn.Module):
    def __init__(self,):
        super(SincNet,self).__init__()
        
        
        
        input_dim =101951
        self.sinc_conv=SincConv_fast(80,251).cuda() # 80필터 , 251 kernel size
        self.ln0 = LayerNorm(input_dim)
        
        current_dim = input_dim-251+1 # sinc 후
        current_dim = int((current_dim-2-1)/3 +1) # maxpool 1 후
        self.ln1= LayerNorm(current_dim)
        self.active1 = nn.LeakyReLU(0.2)
        self.dropout1 = nn.Dropout(p=0.0)
        
        #### 여기까지1번레이어 ##
        
        self.conv1 = nn.Conv1d(80,60,5)
        current_dim = current_dim - 5 + 1
        current_dim = int((current_dim-2-1)/3+1) # maxpool 2 후
        self.ln2 = LayerNorm(current_dim)
        self.active2 = nn.LeakyReLU(0.2)
        self.dropout2 = nn.Dropout(p=0.0)
        
        
        
        ### 2번 레이어###
        self.conv2 = nn.Conv1d(60,30,5)
        current_dim = current_dim - 5 + 1
        current_dim = int((current_dim-2-1)/3+1) # maxpool 2 후
        self.ln3 = LayerNorm(current_dim)
        self.active3 = nn.LeakyReLU(0.2)
        self.dropout3 = nn.Dropout(p=0.0)        
        
        
        ### 3번 레이어###
        self.conv3 = nn.Conv1d(30,10,5)
        current_dim = current_dim - 5 + 1
        current_dim = int((current_dim-2-1)/3+1) # maxpool 2 후
        self.ln4 = LayerNorm(current_dim)
        self.active4 = nn.LeakyReLU(0.2)
        self.dropout4 = nn.Dropout(p=0.7)           

        ### 4번 레이어###
        self.conv4 = nn.Conv1d(10,1,5)
        current_dim = current_dim - 5 + 1
        current_dim = int((current_dim-2-1)/3+1) # maxpool 2 후
        self.ln5 = LayerNorm(current_dim)
        self.active5 = nn.LeakyReLU(0.2)
        self.dropout5 = nn.Dropout(p=0.7)          
        
        
        #self.avgpool_1 = nn.AdaptiveAvgPool1d(50)
        
        
        
        current_dim= 416 # view 후에
        self.fc = nn.Sequential(nn.Linear(416,256),
                                nn.BatchNorm1d(256),
                                nn.ReLU(),
                                nn.Dropout(p=0.7),
                                nn.Linear(256,2),
                               )
        
        
        
    def forward(self,x):
        x = self.ln0(x)
        x = self.sinc_conv(x)
        x = F.max_pool1d(torch.abs(x),kernel_size=3,stride=3)
        x = self.ln1(x)
        x = self.active1(x)
        #x = self.dropout1(x)
        
        ### 1번###
        x = self.conv1(x)
        x = F.max_pool1d(x,kernel_size=3,stride=3)
        x = self.ln2(x)
        x = self.active2(x)
        #x = self.dropout2(x)
        
        ### 2번 ###
        x = self.conv2(x)
        x = F.max_pool1d(x,kernel_size=3,stride=3)
        x = self.ln3(x)
        x = self.active3(x)
        #x = self.dropout3(x)        
        
        ### 3번 ###
        x = self.conv3(x)
        x = F.max_pool1d(x,kernel_size=3,stride=3)
        x = self.ln4(x)
        x = self.active4(x)
        #x = self.dropout3(x) 
        
        #print(x.size())
        ### 4번 ###
        x = self.conv4(x)
        x = F.max_pool1d(x,kernel_size=3,stride=3)
        x = self.ln5(x)
        x = self.active5(x)
        #x = self.dropout3(x) 
        
        
        #print(x.size())
        #x = self.avgpool(x)
        x = x.view(-1,416)
        x = self.fc(x)
        
        
        return x 
        
        

        
# 전체 레이어
# 순서: sinc -> pooling -> layer norm -> leaky relu -> drop out -> cnn/dnn layer -> softmax




In [13]:
#7. Optimizer, Objective Function
def model_initialize():
    model = SincNet().to(DEVICE)
    #원핫 인코딩값의 loss는 crossEntropyLoss로 비교
    #print(model)
    return model

In [14]:
#8. 학습
def train(model,train_loader,optimizer, log_interval):
    model.train()
    correct = 0
    train_loss = 0
    for batch_idx,(image,label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        #데이터들 장비에 할당
        optimizer.zero_grad() # device 에 저장된 gradient 제거
        output = model(image) # model로 output을 계산
        loss = criterion(output, label) #loss 계산
        train_loss += loss.item()
        prediction = output.max(1,keepdim=True)[1] # 가장 확률이 높은 class 1개를 가져온다.그리고 인덱스만
        correct += prediction.eq(label.view_as(prediction)).sum().item()# 아웃풋이 배치 사이즈 32개라서.
        loss.backward() # loss 값을 이용해 gradient를 계산
        optimizer.step() # Gradient 값을 이용해 파라미터 업데이트.
    train_loss/=len(train_loader.dataset)
    train_accuracy = 100. * correct / len(train_loader.dataset)
    return train_loss,train_accuracy


In [15]:
#9. 학습 진행하며, validation 데이터로 모델 성능확인
def evaluate(model,valid_loader):
    model.eval()
    valid_loss = 0
    correct = 0
    #no_grad : 그래디언트 값 계산 막기.
    with torch.no_grad():
        for image, label in valid_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            valid_loss += criterion(output, label).item()
            prediction = output.max(1,keepdim=True)[1] # 가장 확률이 높은 class 1개를 가져온다.그리고 인덱스만
            correct += prediction.eq(label.view_as(prediction)).sum().item()# 아웃풋이 배치 사이즈 32개라서.
            #true.false값을 sum해줌. item
        valid_loss /= len(valid_loader.dataset)
        valid_accuracy = 100. * correct / len(valid_loader.dataset)
        return valid_loss,valid_accuracy


In [16]:
#데이터 로더 제작 함수

def load_data(data_ind):

    train_loader = torch.utils.data.DataLoader(dataset = 
                                               svd_dataset(
                                                   X_train_list,
                                                   classes,
                                                   data_num=data_ind,
                                                   training=True
                                               ),
                                               batch_size = BATCH_SIZE,
                                               shuffle = True,
                                               ) # 순서가 암기되는것을 막기위해.

    validation_loader = torch.utils.data.DataLoader(dataset = 
                                               svd_dataset(
                                                   X_valid_list,
                                                   classes,
                                                   data_num=data_ind,
                                                   training=False
                                               ),
                                               batch_size = BATCH_SIZE,
                                               shuffle = True,) 
    return train_loader,validation_loader



In [17]:
#10. 학습 및 평가.
# resnet34 pretrained true
# kfold 적용

train_accs = []
valid_accs = []

for data_ind in range(1,6):

    check_path = '../checkpoint/checkpoint_sinc_'+str(data_ind)+'.pt'
    print(check_path)
    early_stopping = EarlyStopping(patience = 5, verbose = True, path=check_path)
    train_loader,validation_loader = load_data(data_ind-1)
    
    best_train_acc=0 # accuracy 기록용
    best_valid_acc=0
    
    model=model_initialize()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)
    
    
    print("[{} 교차검증] 학습 시작\n ----- ".format(data_ind))
    for Epoch in range(1,EPOCHS+1):
        train_loss,train_accuracy=train(model,train_loader,optimizer,log_interval=31)
        valid_loss,valid_accuracy = evaluate(model, validation_loader)


        print("\n[EPOCH:{}]\t Train Loss:{:.4f}\t Train Acc:{:.2f} %  | \tValid Loss:{:.4f} \tValid Acc: {:.2f} %\n".
              format(Epoch,train_loss,train_accuracy,valid_loss,valid_accuracy))
        

        early_stopping(valid_loss, model)
        if -early_stopping.best_score == valid_loss:
            best_train_acc, best_valid_acc = train_accuracy,valid_accuracy
        
        if early_stopping.early_stop:
                train_accs.append(best_train_acc)
                valid_accs.append(best_valid_acc)
                print("[{} 교차검증] Early stopping".format(data_ind))
                break

        if Epoch==EPOCHS:
            #만약 early stop 없이 40 epoch라서 중지 된 경우.
            train_accs.append(best_train_acc)
            valid_accs.append(best_valid_acc)

../checkpoint/checkpoint_sinc_1.pt
[1 교차검증] 학습 시작
 ----- 

[EPOCH:1]	 Train Loss:0.0225	 Train Acc:58.13 %  | 	Valid Loss:0.0189 	Valid Acc: 68.94 %

Validation loss decreased (inf --> 0.018938).  Saving model ...

[EPOCH:2]	 Train Loss:0.0185	 Train Acc:71.36 %  | 	Valid Loss:0.0184 	Valid Acc: 71.67 %

Validation loss decreased (0.018938 --> 0.018382).  Saving model ...

[EPOCH:3]	 Train Loss:0.0161	 Train Acc:75.62 %  | 	Valid Loss:0.0186 	Valid Acc: 72.35 %

EarlyStopping counter: 1 out of 5

[EPOCH:4]	 Train Loss:0.0138	 Train Acc:80.21 %  | 	Valid Loss:0.0169 	Valid Acc: 75.77 %

Validation loss decreased (0.018382 --> 0.016890).  Saving model ...

[EPOCH:5]	 Train Loss:0.0126	 Train Acc:82.96 %  | 	Valid Loss:0.0169 	Valid Acc: 72.35 %

Validation loss decreased (0.016890 --> 0.016871).  Saving model ...

[EPOCH:6]	 Train Loss:0.0106	 Train Acc:86.63 %  | 	Valid Loss:0.0159 	Valid Acc: 77.47 %

Validation loss decreased (0.016871 --> 0.015934).  Saving model ...

[EPOCH:7]	 Trai

In [18]:
sum_valid=0
for data_ind in range(5):
    print("[{} 교차검증] train ACC : {:.4f} |\t valid ACC: {:.4f} ".format(data_ind+1,train_accs[data_ind],valid_accs[data_ind] ))
    sum_valid+=valid_accs[data_ind]
    
print("평균 검증 정확도",sum_valid/5,"%")

[1 교차검증] train ACC : 94.6920 |	 valid ACC: 78.8396 
[2 교차검증] train ACC : 70.5111 |	 valid ACC: 76.3699 
[3 교차검증] train ACC : 88.0079 |	 valid ACC: 76.3699 
[4 교차검증] train ACC : 86.8283 |	 valid ACC: 74.6575 
[5 교차검증] train ACC : 80.2356 |	 valid ACC: 72.9452 
평균 검증 정확도 75.83641123942213 %


In [20]:
#9. 학습 진행하며, validation 데이터로 모델 성능확인
def test_evaluate(model,test_loader):
    model.eval()
    predictions = []
    answers = []
    #no_grad : 그래디언트 값 계산 막기.
    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            output = F.softmax(output, dim=1).data.squeeze() # softmax 적용 (모델을 통과는 했지만, criterion는 안통과함.)
            prediction = output.max(1,keepdim=True)[1] # 가장 확률이 높은 class 1개를 가져온다.그리고 인덱스만
            answers +=label
            predictions +=prediction
        return predictions,answers

In [21]:
# Confusion matrix (resnet18)
# kfold의 confusion matrix는 계산 방법이 다르다.
# 모델을 각각 불러와서 test set을 평가한다.

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

cf = np.zeros((2,2))
cf_list = []
average_accuracy = 0
average_fscore = 0

for data_ind in range(1,6):

    check_path = '../checkpoint/checkpoint_sinc_'+str(data_ind)+'.pt'
    model.load_state_dict(torch.load(check_path))

    predictions,answers = test_evaluate(model, test_loader)
    predictions=[ dat.cpu().numpy() for dat in predictions]
    answers=[ dat.cpu().numpy() for dat in answers]

    
    cf = confusion_matrix(answers, predictions)
    cf_list.append(cf)
    
    acc = (cf[0,0]+cf[1,1])/(cf[0,0]+cf[0,1]+cf[1,0]+cf[1,1])
    average_accuracy+=acc
    precision=cf[0,0]/(cf[0,0]+cf[1,0])
    recall=cf[0,0]/(cf[0,0]+cf[0,1])
    #fscore=2*precision*recall/(precision+recall)
    
    #fscroe macro추가
    fscore = f1_score(answers,predictions,average='macro')
    average_fscore+=fscore
    
    print('{}번 모델'.format(data_ind))
    print("Accuracy : {:.4f}% ".format(acc*100))
    print("Precision (pathology 예측한 것중 맞는 것) : {:.4f}".format(precision))
    print("recall (실제 pathology 중  예측이 맞는 것) : {:.4f}".format(recall))
    print("f score : {:.4f} ".format(fscore))
    print(cf)
    print("-----")

print("평균 acc : {:.4f}".format(average_accuracy/5))
print("평균 f1score : {:.4f}".format(average_fscore/5))




1번 모델
Accuracy : 81.6940% 
Precision (pathology 예측한 것중 맞는 것) : 0.8333
recall (실제 pathology 중  예측이 맞는 것) : 0.8996
f score : 0.7900 
[[215  24]
 [ 43  84]]
-----
2번 모델
Accuracy : 71.5847% 
Precision (pathology 예측한 것중 맞는 것) : 0.8426
recall (실제 pathology 중  예측이 맞는 것) : 0.6946
f score : 0.7051 
[[166  73]
 [ 31  96]]
-----
3번 모델
Accuracy : 77.8689% 
Precision (pathology 예측한 것중 맞는 것) : 0.7992
recall (실제 pathology 중  예측이 맞는 것) : 0.8828
f score : 0.7426 
[[211  28]
 [ 53  74]]
-----
4번 모델
Accuracy : 77.0492% 
Precision (pathology 예측한 것중 맞는 것) : 0.8270
recall (실제 pathology 중  예측이 맞는 것) : 0.8201
f score : 0.7477 
[[196  43]
 [ 41  86]]
-----
5번 모델
Accuracy : 78.1421% 
Precision (pathology 예측한 것중 맞는 것) : 0.8472
recall (실제 pathology 중  예측이 맞는 것) : 0.8117
f score : 0.7630 
[[194  45]
 [ 35  92]]
-----
평균 acc : 0.7727
평균 f1score : 0.7497
